#Mapeamento de médias do ENEM por estado com Folium

#### Objetivo: Mapear todas as médias do ENEM por escola, isto é, vamos colocar um pin em cada localização de uma unidade escolar a sua respectiva média.
---




Criado por Wendel Marques

*   [GitHub](https://www.github.com/WendelMarques)
*   [LinkedIn](https://www.linkedin.com/in/wendelmarques/)
*   [Medium](https://medium.com/@WendelMarquesJS/mapeamento-de-m%C3%A9dias-do-enem-por-estado-com-folium-bf61fe23a3d8?source=your_stories_page---------------------------)




# Bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import json
import folium
from folium import Marker
from folium.plugins import MarkerCluster
from jinja2 import Template
from branca.element import Template, MacroElement

# Tratamento dos dados do ENEM


## Leitura dos dados e visualização do conteúdo

In [ ]:
#carrega os dados no dataframe
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/enem/microdados-enem-escola.csv', encoding = 'latin-1', sep=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#vizualiza as primeiras 5 entradas do dataframe
df.head()

,NU_ANO,CO_UF_ESCOLA,SG_UF_ESCOLA,CO_MUNICIPIO_ESCOLA,NO_MUNICIPIO_ESCOLA,CO_ESCOLA_EDUCACENSO,NO_ESCOLA_EDUCACENSO,TP_DEPENDENCIA_ADM_ESCOLA,TP_LOCALIZACAO_ESCOLA,NU_MATRICULAS,NU_PARTICIPANTES_NEC_ESP,NU_PARTICIPANTES,NU_TAXA_PARTICIPACAO,NU_MEDIA_CN,NU_MEDIA_CH,NU_MEDIA_LP,NU_MEDIA_MT,NU_MEDIA_RED,NU_MEDIA_OBJ,NU_MEDIA_TOT,INSE,PC_FORMACAO_DOCENTE,NU_TAXA_PERMANENCIA,NU_TAXA_APROVACAO,NU_TAXA_REPROVACAO,NU_TAXA_ABANDONO,PORTE_ESCOLA
0,2007,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,144,NaN,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.03,NaN,NaN,NaN,91.9,8.1,0.0,Maior que 90 alunos
1,2006,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,184,NaN,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.82,NaN,NaN,NaN,NaN,NaN,NaN,Maior que 90 alunos
2,2005,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,220,NaN,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.83,NaN,NaN,NaN,86.5,12.4,1.1,Maior que 90 alunos
3,2008,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,186,NaN,171,NaN,NaN,NaN,NaN,NaN,72.16,60.02,NaN,NaN,NaN,NaN,90.3,9.7,0.0,Maior que 90 alunos
4,2007,11,RO,1100205,Porto Velho,11000171,CENTRO EDUCACIONAL MOJUCA,4,1,19,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.84,NaN,NaN,NaN,74.2,21.0,4.8,De 1 a 30 alunos


In [ ]:
#imprime o numero de linhas e colunas do dataframe
df.shape

(172305, 27)

In [ ]:
#imprime o tipo de dado em cada coluna
df.dtypes

NU_ANO                         int64
CO_UF_ESCOLA                   int64
SG_UF_ESCOLA                  object
CO_MUNICIPIO_ESCOLA            int64
NO_MUNICIPIO_ESCOLA           object
CO_ESCOLA_EDUCACENSO           int64
NO_ESCOLA_EDUCACENSO          object
TP_DEPENDENCIA_ADM_ESCOLA      int64
TP_LOCALIZACAO_ESCOLA          int64
NU_MATRICULAS                  int64
NU_PARTICIPANTES_NEC_ESP     float64
NU_PARTICIPANTES               int64
NU_TAXA_PARTICIPACAO         float64
NU_MEDIA_CN                  float64
NU_MEDIA_CH                  float64
NU_MEDIA_LP                  float64
NU_MEDIA_MT                  float64
NU_MEDIA_RED                 float64
NU_MEDIA_OBJ                 float64
NU_MEDIA_TOT                 float64
INSE                          object
PC_FORMACAO_DOCENTE          float64
NU_TAXA_PERMANENCIA          float64
NU_TAXA_APROVACAO            float64
NU_TAXA_REPROVACAO           float64
NU_TAXA_ABANDONO             float64
PORTE_ESCOLA                  object
d

In [ ]:
#lista o nome das colunas do dataframe
list(df.columns)

['NU_ANO',
 'CO_UF_ESCOLA',
 'SG_UF_ESCOLA',
 'CO_MUNICIPIO_ESCOLA',
 'NO_MUNICIPIO_ESCOLA',
 'CO_ESCOLA_EDUCACENSO',
 'NO_ESCOLA_EDUCACENSO',
 'TP_DEPENDENCIA_ADM_ESCOLA',
 'TP_LOCALIZACAO_ESCOLA',
 'NU_MATRICULAS',
 'NU_PARTICIPANTES_NEC_ESP',
 'NU_PARTICIPANTES',
 'NU_TAXA_PARTICIPACAO',
 'NU_MEDIA_CN',
 'NU_MEDIA_CH',
 'NU_MEDIA_LP',
 'NU_MEDIA_MT',
 'NU_MEDIA_RED',
 'NU_MEDIA_OBJ',
 'NU_MEDIA_TOT',
 'INSE',
 'PC_FORMACAO_DOCENTE',
 'NU_TAXA_PERMANENCIA',
 'NU_TAXA_APROVACAO',
 'NU_TAXA_REPROVACAO',
 'NU_TAXA_ABANDONO',
 'PORTE_ESCOLA']

## Remoção de colunas

In [ ]:
#remove colunas desnecessárias
df.drop(['TP_DEPENDENCIA_ADM_ESCOLA',
         'TP_LOCALIZACAO_ESCOLA',
         'NU_MATRICULAS',
         'NU_PARTICIPANTES_NEC_ESP',
         'NU_PARTICIPANTES',
         'NU_TAXA_PARTICIPACAO'], axis=1,inplace=True)

In [ ]:
#remove mais colunas
#como a analise leva em consideracao os exames de 2009 a 2015
#as colunas 'NU_MEDIA_OBJ','NU_MEDIA_TOT' são desnecessárias
#levando em conta a alteracao do formato da prova
df.drop(['NU_MEDIA_TOT',
         'NU_MEDIA_OBJ',
         'INSE',
         'PC_FORMACAO_DOCENTE',
         'NU_TAXA_PERMANENCIA',
         'NU_TAXA_APROVACAO',
         'NU_TAXA_REPROVACAO',
         'NU_TAXA_ABANDONO',
         'PORTE_ESCOLA'], axis=1, inplace=True)

In [ ]:
df.shape

(172305, 12)

## Tratamento de dados faltantes  

In [ ]:
#verificando se existem valores faltantes nos dados
df.isna().any()

NU_ANO                  False
CO_UF_ESCOLA            False
SG_UF_ESCOLA            False
CO_MUNICIPIO_ESCOLA     False
NO_MUNICIPIO_ESCOLA     False
CO_ESCOLA_EDUCACENSO    False
NO_ESCOLA_EDUCACENSO    False
NU_MEDIA_CN              True
NU_MEDIA_CH              True
NU_MEDIA_LP              True
NU_MEDIA_MT              True
NU_MEDIA_RED             True
dtype: bool

In [ ]:
#verificando se existem valores faltantes nos dados
df.isna().sum()

NU_ANO                      0
CO_UF_ESCOLA                0
SG_UF_ESCOLA                0
CO_MUNICIPIO_ESCOLA         0
NO_MUNICIPIO_ESCOLA         0
CO_ESCOLA_EDUCACENSO        0
NO_ESCOLA_EDUCACENSO        0
NU_MEDIA_CN             67618
NU_MEDIA_CH             67618
NU_MEDIA_LP             67618
NU_MEDIA_MT             67618
NU_MEDIA_RED            48854
dtype: int64

In [ ]:
#descartando as linhas que tem poucos valores faltantes
df.dropna(axis=0, subset=['NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT', 'NU_MEDIA_RED'],inplace=True)

In [ ]:
#verificando quantas linhas de dados ainda estão no dataframe. Perdemos cerca de 39,46% dos dados
df.shape

(104307, 12)

In [ ]:
#verificando novamente se existem valores faltantes nos dados
df.isna().sum()

NU_ANO                  0
CO_UF_ESCOLA            0
SG_UF_ESCOLA            0
CO_MUNICIPIO_ESCOLA     0
NO_MUNICIPIO_ESCOLA     0
CO_ESCOLA_EDUCACENSO    0
NO_ESCOLA_EDUCACENSO    0
NU_MEDIA_CN             0
NU_MEDIA_CH             0
NU_MEDIA_LP             0
NU_MEDIA_MT             0
NU_MEDIA_RED            0
dtype: int64

## Inclusão de média das notas de cada escola

In [ ]:
#calculando a media simples das notas
df['MEDIA TOTAL'] = (df['NU_MEDIA_RED'] + df['NU_MEDIA_MT'] + df['NU_MEDIA_LP'] + df['NU_MEDIA_CN'] + df['NU_MEDIA_CH'])/5




In [ ]:
df.head()

,NU_ANO,CO_UF_ESCOLA,SG_UF_ESCOLA,CO_MUNICIPIO_ESCOLA,NO_MUNICIPIO_ESCOLA,CO_ESCOLA_EDUCACENSO,NO_ESCOLA_EDUCACENSO,NU_MEDIA_CN,NU_MEDIA_CH,NU_MEDIA_LP,NU_MEDIA_MT,NU_MEDIA_RED,MEDIA TOTAL
67618,2014,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,604.88,647.29,592.34,629.14,718.53,638.436
67619,2015,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,591.64,652.34,604.53,627.66,732.00,641.634
67620,2009,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,622.92,624.16,598.73,592.42,707.29,629.104
67621,2011,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,589.18,587.03,617.56,683.44,649.24,625.290
67622,2012,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,587.74,624.71,576.79,660.32,653.02,620.516


In [ ]:
# #removendo linhas de dados de anos anteriores a 2009
# df.drop(df[df.NU_ANO < 2009].index, inplace=True)

In [ ]:
#altera index para 0 - len(df)
indices = list(range(0, len(df)))
df.index = indices

In [ ]:

df = df.groupby(['CO_ESCOLA_EDUCACENSO'], as_index=False)['MEDIA TOTAL'].mean()
df.head()

,CO_ESCOLA_EDUCACENSO,MEDIA TOTAL
0,11000058,631.583429
1,11000171,507.097000
2,11000180,502.575333
3,11000198,570.662571
4,11000244,562.182000


In [ ]:
#salvando df
df.to_csv('tratamento-dados-enem.csv', index=False)

# Tratamento dados de localização das escolas

In [ ]:
listaEscolas = pd.read_csv('/content/drive/My Drive/Colab Notebooks/enem/Análise - Tabela da lista das escolas - Detalhado.csv', encoding = 'utf-8', sep=';')

In [ ]:
listaEscolas.head()

,Restrição de Atendimento,Escola,Código INEP,UF,Município,Localização,Localidade Diferenciada,Categoria Administrativa,Endereço,Telefone,Dependência Administrativa,Categoria Escola Privada,Conveniada Poder Público,Regulamentação pelo Conselho de Educação,Porte da Escola,Etapas e Modalidade de Ensino Oferecidas,Outras Ofertas Educacionais,Latitude,Longitude
0,ESCOLA ATENDE EXCLUSIVAMENTE ALUNOS COM DEFICI...,EEEE ABNAEL MACHADO DE LIMA - CENE,11000023,RO,Porto Velho,Urbana,A escola não está em área de localização difer...,Pública,"AVENIDA AMAZONAS, 6492 ZONA LESTE. TIRADENTES....",(69) 992083054,Estadual,Não Informado,Não,Não,Entre 51 e 200 matrículas de escolarização,Ensino Fundamental,Atendimento Educacional Especializado,-8.758459,-63.854011
1,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EMEIEF PEQUENOS TALENTOS,11000040,RO,Porto Velho,Urbana,A escola não está em área de localização difer...,Pública,"RUA CAETANO, 3256 PREDIO. CALADINHO. 76808-108...",(69) 32135237,Municipal,Não Informado,Não,Sim,Entre 201 e 500 matrículas de escolarização,Educação Infantil,NaN,-8.793730,-63.883919
2,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,CENTRO DE ENSINO CLASSE A,11000058,RO,Porto Velho,Urbana,A escola não está em área de localização difer...,Privada,"AVENIDA CARLOS GOMES, 1135 CENTRO. 76801-123 P...",(69) 32244473,Privada,Particular,Não,Sim,Mais de 1000 matrículas de escolarização,"Educação Infantil, Ensino Fundamental, Ensino ...",NaN,-8.760734,-63.901986
3,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,CENTRO EDUCACIONAL PRESBITERIANO 15 DE NOVEMBRO,11000082,RO,Porto Velho,Urbana,A escola não está em área de localização difer...,Privada,"RUA ALMIRANTE BARROSO, 1483 SANTA BARBARA. 768...",(69) 32245636,Privada,Particular,Não,Sim,Entre 51 e 200 matrículas de escolarização,"Educação Infantil, Ensino Fundamental",NaN,-8.765205,-63.896177
4,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,CENTRO EDUC CORA CORALINA,11000104,RO,Porto Velho,Urbana,A escola não está em área de localização difer...,Privada,"RUA MEXICO, 1056 NOVA PORTO VELHO. 76820-190 P...",(69) 32252616,Privada,Particular,Não,Sim,Entre 501 e 1000 matrículas de escolarização,"Educação Infantil, Ensino Fundamental",NaN,-8.768632,-63.875471


In [ ]:
#exclusao de colunas que não serão utilizadas
listaEscolas.drop(['Localização',	'Localidade Diferenciada',	'Categoria Administrativa',	
                   'Endereço',	'Telefone',	'Dependência Administrativa',	'Categoria Escola Privada',
                   'Conveniada Poder Público',	'Regulamentação pelo Conselho de Educação',	'Porte da Escola',
                   'Etapas e Modalidade de Ensino Oferecidas', 'Outras Ofertas Educacionais', 'Restrição de Atendimento'], 
                    axis = 1, inplace=True)

In [ ]:
listaEscolas.head()

,Escola,Código INEP,UF,Município,Latitude,Longitude
0,EEEE ABNAEL MACHADO DE LIMA - CENE,11000023,RO,Porto Velho,-8.758459,-63.854011
1,EMEIEF PEQUENOS TALENTOS,11000040,RO,Porto Velho,-8.793730,-63.883919
2,CENTRO DE ENSINO CLASSE A,11000058,RO,Porto Velho,-8.760734,-63.901986
3,CENTRO EDUCACIONAL PRESBITERIANO 15 DE NOVEMBRO,11000082,RO,Porto Velho,-8.765205,-63.896177
4,CENTRO EDUC CORA CORALINA,11000104,RO,Porto Velho,-8.768632,-63.875471


In [ ]:
#verificando dados faltantes 
listaEscolas.isna().sum()

Escola             0
Código INEP        0
UF                 0
Município          0
Latitude       69786
Longitude      69786
dtype: int64

In [ ]:
listaEscolas.shape

(222936, 6)

In [ ]:
#descartando linhas que tem dados faltantes
listaEscolas.dropna(axis=0, subset=['Latitude', 'Longitude'], inplace=True)

In [ ]:
#verificando quantas linhas de dados ainda estão no dataframe. Perdemos cerca de 31% dos dados
listaEscolas.shape

(153150, 6)

In [ ]:
#salvando df
listaEscolas.to_csv('tratamento-dados-localizacao.csv', index=False)

# Merge dados do ENEM e localização das escolas

In [ ]:
#converte coluna "Codigo INEP" em indice
listaEscolas.index = listaEscolas['Código INEP']

In [ ]:
#excluindo coluna 'Código INEP'
listaEscolas.drop('Código INEP', axis=1, inplace=True)

In [ ]:
listaEscolas.head()

,Escola,UF,Município,Latitude,Longitude
Código INEP,,,,,
11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,RO,Porto Velho,-8.758459,-63.854011
11000040,EMEIEF PEQUENOS TALENTOS,RO,Porto Velho,-8.793730,-63.883919
11000058,CENTRO DE ENSINO CLASSE A,RO,Porto Velho,-8.760734,-63.901986
11000082,CENTRO EDUCACIONAL PRESBITERIANO 15 DE NOVEMBRO,RO,Porto Velho,-8.765205,-63.896177
11000104,CENTRO EDUC CORA CORALINA,RO,Porto Velho,-8.768632,-63.875471


In [ ]:
df.head()

,CO_ESCOLA_EDUCACENSO,MEDIA TOTAL
0,11000058,631.583429
1,11000171,507.097000
2,11000180,502.575333
3,11000198,570.662571
4,11000244,562.182000


In [ ]:
df.shape

(25022, 2)

In [ ]:
mergeEscolaLocalizacao = df.join(listaEscolas, on='CO_ESCOLA_EDUCACENSO')

In [ ]:
mergeEscolaLocalizacao.head()

,CO_ESCOLA_EDUCACENSO,MEDIA TOTAL,Escola,UF,Município,Latitude,Longitude
0,11000058,631.583429,CENTRO DE ENSINO CLASSE A,RO,Porto Velho,-8.760734,-63.901986
1,11000171,507.097000,CENTRO EDUCACIONAL MOJUCA,RO,Porto Velho,-8.765028,-63.891857
2,11000180,502.575333,NaN,NaN,NaN,NaN,NaN
3,11000198,570.662571,COLEGIO SAPIENS - UNIDADE JARDIM DAS MANGUEIRAS,RO,Porto Velho,-8.752404,-63.868376
4,11000244,562.182000,COLEGIO DOM BOSCO,RO,Porto Velho,-8.766677,-63.900223


In [ ]:
mergeEscolaLocalizacao.shape 

(25022, 7)

In [ ]:
df.shape

(25022, 2)

In [ ]:
#excluindo dados faltantes
mergeEscolaLocalizacao.dropna(axis=0, subset=['Longitude', 'Latitude'], inplace=True)
mergeEscolaLocalizacao.isna().sum()

CO_ESCOLA_EDUCACENSO    0
MEDIA TOTAL             0
Escola                  0
UF                      0
Município               0
Latitude                0
Longitude               0
dtype: int64

In [ ]:
#salvando merge
mergeEscolaLocalizacao.to_csv('merge-df-e-listaEscolas.csv', index=False)

#Plotagem do mapa



## Criação do Mapa

In [ ]:

#"Passagem de propriedades personalizadas via marcador, 
#no Folium o marcador padrão não o suporta, mas a seguinte classe de 
#marcador pode ser introduzida, estendendo a classe Marker" (fonte: Vadim Gremyachev/Stack OverFlow)

class MarkerWithProps(Marker):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
        var {{this.get_name()}} = L.marker(
            [{{this.location[0]}}, {{this.location[1]}}],
            {
                icon: new L.Icon.Default(),
                {%- if this.draggable %}
                draggable: true,
                autoPan: true,
                {%- endif %}
                {%- if this.props %}
                props : {{ this.props }} 
                {%- endif %}
                }
            )
            .addTo({{this._parent.get_name()}});
        {% endmacro %}
        """)
    def __init__(self, location, popup=None, tooltip=None, icon=None,name=None,
                 draggable=False, props = None ):
        super(MarkerWithProps, self).__init__(location=location,popup=popup,tooltip=tooltip,icon=icon,draggable=draggable)
        self.props = json.loads(json.dumps(props))    


# Personalização da função icon_create_function do cluster de marcadores,
# para substituição do rótulo do marcador para exibir o valor personalizado em vez do padrão 
# (contagem de marcadores em um cluster) (fonte: Vadim Gremyachev/Stack OverFlow)

icon_create_function = '''
    function(cluster) {
        var markers = cluster.getAllChildMarkers();
        var sum = 0;
        for (var i = 0; i < markers.length; i++) {
            sum += markers[i].options.props.mediaTotal;
        }
        var avg = sum/cluster.getChildCount();   
        avg=avg.toFixed(2);  

        function verifica_Media(media) {
            if (media < 520) {
                return 'marker-cluster marker-cluster-large'
            }
            else if (media >=  520 && media <= 600) {
                return 'marker-cluster marker-cluster-medium'
            }
            else  if (media > 600) {
                return 'marker-cluster marker-cluster-small'
            }
        }

        return L.divIcon({
             html: '<div style="display:flex;justify-content:center;align-items:center;font-size:7pt;">'+ avg +'</div>',
             className: verifica_Media(avg),
             iconSize: new L.Point(40, 40)
        });
    }
'''

## CRIAÇÃO DO MAPA
width, height = 480, 350
from branca.element import Figure
fig = Figure(width=width, height=height)

map = folium.Map(location=[-9.026078, -70.441312], zoom_start=4, tiles=None)
folium.TileLayer(name='ENEM MÉDIAS').add_to(map)

maker_data_list = []
for index, row in mergeEscolaLocalizacao.iterrows():
        maker_data_list.append({'localizacao':[row['Latitude'], row['Longitude']], 'mediaTotal': round(row['MEDIA TOTAL'], 2), 
                                'nomeEscola': row['Escola'], 'UF': row['UF']})

marker_data = tuple(maker_data_list)

estados = {}
grupos = {}
estados_unique = np.unique(mergeEscolaLocalizacao['UF'])

for uf in estados_unique:
    estados[uf] = MarkerCluster(icon_create_function=icon_create_function, name=uf)
    grupos[uf] = folium.FeatureGroup(name=uf)

for uf in estados:
    for marker_item in marker_data:
        if (marker_item['UF'] == uf):
            MarkerWithProps(
                    location=marker_item['localizacao'],
                    props = {'mediaTotal': marker_item['mediaTotal']},
                    popup=marker_item['nomeEscola'] + ":<br>" + '<b>' + str(marker_item['mediaTotal']) + '</b>',
                    icon =  (
                            folium.Icon(color='red', icon_color='#f5f6fa', icon='university', prefix='fa') 
                            if marker_item['mediaTotal'] < 600 
                            else folium.Icon(color='blue', icon_color='#f5f6fa',
                                            icon='university', prefix='fa')
                            ),
                ).add_to(estados[uf])

    grupos[uf].add_child(estados[uf])
    estados[uf].add_to(map)
            

## Legenda

In [ ]:

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="utf-8">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Mapeamento de médias do ENEM por escola (2009 - 2015)</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

</head>
<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:25px; left: 0px; top: 0px;'>
     
    <div class='legend-title'><font size="2">Mapeamento de médias do ENEM</div>
    <div class='legend-scale'><font size="1">Provas de 2009 a 2015 | github.com/WendelMarques</font></div>
</div>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid white; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:11px; left: 20px; bottom: 20px;'>
     
<div class='legend-title'>Agrupamentos</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:orange;opacity:0.7;'></span> 339 - 520</li>
    <li><span style='background:yellow;opacity:0.7;'></span>520 - 600</li>
    <li><span style='background:green;opacity:0.7;'></span>600 - 753</li>

  </ul>
  <div class='legend-title'>Marcadores</div>
    <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Menor que 600 </li>
    <li><span style='background:blue;opacity:0.7;'></span>Maior que 600</li>
  </ul>

  </ul>
  <div class='legend-title'></div>
    

</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

map.get_root().add_child(macro)

## Salva mapa

In [ ]:
folium.LayerControl('topright', collapsed=True).add_to(map)
# map.save('/content/drive/My Drive/Colab Notebooks/mapeamento-enem/mapa-completo.html')
map.save('mapa-completo.html')